In [1]:
using Conda, Pkg, PyCall, Random

In [14]:
Conda.add("cryptography", :fernet)
ENV["PYTHON"] = joinpath(Conda.ROOTENV, "envs/fernet/bin/python")
Pkg.build("PyCall")

┌ Info: Running `conda install -y cryptography` in fernet environment
└ @ Conda /home/kon/.julia/packages/Conda/x2UxR/src/Conda.jl:127


done
Solving environment: ...working... 

done



# All requested packages already installed.



    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/53b8b07b721b77144a0fbbbc2675222ebf40a02d/build.log`


In [3]:
function gen_key()
    py"""
    from cryptography.fernet import Fernet
    key = Fernet.generate_key()
    """
    py"key"
end

gen_key (generic function with 1 method)

In [4]:
function encrypt(txts::Vector{String}, key::String)
    py"""
    from cryptography.fernet import Fernet
    f = Fernet($key)
    enc = [f.encrypt(bytes(t, 'utf-8')) for t in $txts]
    """
    return py"enc"
end

encrypt (generic function with 1 method)

In [5]:
function decrypt(tokens::Vector{String}, key::String)
    py"""
    from cryptography.fernet import Fernet
    f = Fernet($key)
    dec = [f.decrypt(t) for t in $tokens]
    """
    return py"dec"
end

decrypt (generic function with 1 method)

In [6]:
n_sample = 5
# n_sample = 17*10^4 # encrytion/decryption takes 5 seconds 

5

In [7]:
txts = [randstring("吾輩は猫である。名前はまだない。", 100) for _ in 1:n_sample]

5-element Vector{String}:
 "前輩名ははで吾前は猫いだで。でま吾吾な。名なでだいまで。でる。" ⋯ 117 bytes ⋯ "だなる名。前で。ない。る。はあい。輩はいるは名猫は名前で猫で"
 "。まない輩。ま猫名前だ猫吾名前い猫猫前な。ま吾はい猫はでるまな" ⋯ 117 bytes ⋯ "だ。猫前ははあは。でだは。名前は名名は輩な前吾吾は吾名。だ。"
 "吾はい前だまな。。。な名。。名はなだなだだいは猫るまる猫。輩名" ⋯ 117 bytes ⋯ "。だ。るで猫だあ前名あでは輩名輩で。輩名輩あ吾だ名。猫な前。"
 "いま。はあ前あまま名ま。前ま輩はだあな。名は輩な。吾はる。まな" ⋯ 117 bytes ⋯ "。猫は吾はははるはだはで。だいは前吾猫なま。猫まなは輩。はる"
 "前前。るる。名猫吾は前いは。では吾い前。猫なあまない。名い前前" ⋯ 117 bytes ⋯ "名輩猫まで吾は。で吾ま吾。あだ吾猫ま前吾ま。だはは吾で。い前"

In [8]:
key = gen_key()

"hMIyEXg9mEONwd9DMxm-ypyBURlMqqLDvAsoBCdfRr4="

In [9]:
encs = encrypt(txts, key)

5-element Vector{String}:
 "gAAAAABjnLjtmjXoelUnXnkPFk2kYPN" ⋯ 423 bytes ⋯ "X9Z8Kq6JnInjKBhVwcdyegj2McMg=="
 "gAAAAABjnLjtQWmG0tL-GHgF8-tfwGt" ⋯ 423 bytes ⋯ "YdFi8uTvpYyjjEo02FwsezLgIGZA=="
 "gAAAAABjnLjt3KMpTaRB-VApIAM9JXM" ⋯ 423 bytes ⋯ "v3yfJf_woFGvggTv7a9KJ2yEX4mA=="
 "gAAAAABjnLjt0cwdFr81yUe2JitrpLV" ⋯ 423 bytes ⋯ "BAmT03PGCZ7ZolMGWPIflIWTa3SQ=="
 "gAAAAABjnLjtmWHaDfj-x5cSMuXvw64" ⋯ 423 bytes ⋯ "jRkhSG84l8D7ys3FtEwK6oIwXzog=="

In [10]:
decs = decrypt(encs, key)

5-element Vector{String}:
 "前輩名ははで吾前は猫いだで。でま吾吾な。名なでだいまで。でる。" ⋯ 117 bytes ⋯ "だなる名。前で。ない。る。はあい。輩はいるは名猫は名前で猫で"
 "。まない輩。ま猫名前だ猫吾名前い猫猫前な。ま吾はい猫はでるまな" ⋯ 117 bytes ⋯ "だ。猫前ははあは。でだは。名前は名名は輩な前吾吾は吾名。だ。"
 "吾はい前だまな。。。な名。。名はなだなだだいは猫るまる猫。輩名" ⋯ 117 bytes ⋯ "。だ。るで猫だあ前名あでは輩名輩で。輩名輩あ吾だ名。猫な前。"
 "いま。はあ前あまま名ま。前ま輩はだあな。名は輩な。吾はる。まな" ⋯ 117 bytes ⋯ "。猫は吾はははるはだはで。だいは前吾猫なま。猫まなは輩。はる"
 "前前。るる。名猫吾は前いは。では吾い前。猫なあまない。名い前前" ⋯ 117 bytes ⋯ "名輩猫まで吾は。で吾ま吾。あだ吾猫ま前吾ま。だはは吾で。い前"

In [11]:
@assert all(decs .== txts)